## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI:

```bash
az login
```

# 📈 Azure AI Agent Observability

## Industry Use Case: Customer Service Monitoring

This notebook demonstrates observability for **Azure AI agents** using the AzureAIClient.

| Feature | FSI Application |
|---------|-----------------|
| **Automatic Tracing** | Monitor customer service interactions |
| **Span Context** | Track conversation flow |
| **Application Insights** | Centralized logging |

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../../.env', override=True)

print(f"✅ Environment loaded")
print(f"Project Endpoint: {os.getenv('AI_FOUNDRY_PROJECT_ENDPOINT')[:50]}...")

In [ ]:
import asyncio
from random import uniform

from agent_framework.azure import AzureAIAgentClient
from agent_framework.observability import get_tracer
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry.trace import SpanKind
from opentelemetry.trace.span import format_trace_id

print("✅ All imports loaded")

## Define Customer Service Tools

In [ ]:
async def get_account_info(account_id: str) -> str:
    """Get customer account information."""
    await asyncio.sleep(uniform(0.1, 0.3))  # Simulate DB lookup
    accounts = {
        "ACC-001": {"name": "John Smith", "balance": 15432.50, "status": "Active"},
        "ACC-002": {"name": "Jane Doe", "balance": 8750.25, "status": "Active"},
    }
    info = accounts.get(account_id, {"name": "Unknown", "balance": 0, "status": "Not Found"})
    return f"Account {account_id}: {info['name']}, Balance: ${info['balance']:,.2f}, Status: {info['status']}"


async def create_support_ticket(issue: str, priority: str = "medium") -> str:
    """Create a customer support ticket."""
    await asyncio.sleep(uniform(0.1, 0.2))  # Simulate API call
    ticket_id = f"TKT-{hash(issue) % 10000:04d}"
    return f"Support ticket {ticket_id} created with {priority} priority: {issue[:50]}..."


print("✅ Tools defined: get_account_info, create_support_ticket")

## Run Agent with Observability

The `AzureAIClient` provides automatic telemetry configuration via `configure_azure_monitor()`.

In [ ]:
async def run_customer_service_agent():
    """Run customer service agent with observability."""
    print("\n--- Customer Service Agent with Observability ---\n")
    
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(
            endpoint=os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"],
            credential=credential
        ) as project_client,
    ):
        # Configure Azure Monitor with App Insights from Foundry
        try:
            app_insights_conn = await project_client.telemetry.get_application_insights_connection_string()
            if app_insights_conn:
                configure_azure_monitor(connection_string=app_insights_conn)
                print("✅ Azure Monitor configured with App Insights\n")
            else:
                print("⚠️ No Application Insights configured in Foundry project")
                print("Continuing without Azure Monitor telemetry...\n")
        except Exception as e:
            print(f"⚠️ Could not configure Azure Monitor: {e}")
            print("Continuing without telemetry...\n")
        
        # Create agent using AzureAIAgentClient
        agent = AzureAIAgentClient(credential=credential).as_agent(
            name="CustomerServiceAgent",
            instructions="You are a helpful customer service agent for a bank. Help customers with account inquiries and create support tickets when needed.",
            tools=[get_account_info, create_support_ticket],
        )
        
        # Customer inquiries
        inquiries = [
            "What's the balance on account ACC-001?",
            "I can't access my online banking. Can you create a ticket?",
            "Check account ACC-002 and create a high priority ticket for unauthorized transaction",
        ]
        
        # Run with tracing span
        with get_tracer().start_as_current_span("Customer Service Session", kind=SpanKind.CLIENT) as span:
            trace_id = format_trace_id(span.get_span_context().trace_id)
            print(f"🔍 Trace ID: {trace_id}")
            print("(View in Azure Portal > Application Insights)\n")
            
            thread = agent.get_new_thread()
            for inquiry in inquiries:
                print(f"Customer: {inquiry}")
                print(f"{agent.name}: ", end="")
                async for update in agent.run_stream(inquiry, thread=thread):
                    if update.text:
                        print(update.text, end="")
                print("\n")

await run_customer_service_agent()

## Key Takeaways

| Feature | Description |
|---------|-------------|
| `AzureAIClient` | Provides integrated observability |
| `client.configure_azure_monitor()` | One-line telemetry setup |
| `get_tracer()` | Get current OpenTelemetry tracer |
| `start_as_current_span()` | Create custom trace spans |

## Difference from Foundry Tracing

| Approach | Setup | Use Case |
|----------|-------|----------|
| **Foundry Tracing** | Manual `configure_azure_monitor()` | Custom agents with `OpenAIResponsesClient` |
| **AzureAIClient** | Automatic via `client.configure_azure_monitor()` | Azure AI agents with integrated setup |